# Importing Essential Modules

In [1]:
# To handle datasets
import numpy as np
import pandas as pd

# For visualization
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px

# For statistical tests
from scipy.stats import chi2_contingency, fisher_exact

# Data Processing

## Data Loading

In [2]:
# It is the meta data file which has the whole information regarding the customer data
# Only this file need to be processed to get some more relevant information
data = pd.read_excel("Customer data.xlsx", sheet_name=None)
keys = ['Customer Information', 'Sheet15', 'Package Renewal Data']
[data.pop(key) for key in keys]     # list comprehension
data

{'Aug_22':     Membership Id                  Name  Pkg_Duration  Amt_Paid
 0               1          Navin Sharma             6      4500
 1               2          Nitesh yadav             6      4500
 2               3     Harishankar Shibu             3      2500
 3               4        Jagdish Jadhav             3      4000
 4               5         Mahendra Saha             3      2200
 5               6    Harishankar Nagar              6      4000
 6               7                Kavita             3      3850
 7               8               Abhinav             3      2250
 8               9        Dhankesh Meena             6      4050
 9              10      Gajananda Pareek             3      4100
 10             11  Krishna Kumar Mohbey             3      4200
 11             12      Anuradha Pandey              3      4500
 12             13          Sarath Kumar             3      2250
 13             14      Yash Khandelwal              1       700
 14            

In [3]:
# Keys in the dictionary
data.keys()

dict_keys(['Aug_22', 'Sep_22', 'Oct_22', 'Nov_22', 'Dec_22', 'Jan_23', 'Feb_23', 'Mar_23', 'Apr_23', 'May_24', 'Jun_23', 'Jul_23', 'Aug_23'])

In [4]:
# Aug_22 key values
data["Aug_22"]

,Membership Id,Name,Pkg_Duration,Amt_Paid
0,1,Navin Sharma,6,4500
1,2,Nitesh yadav,6,4500
2,3,Harishankar Shibu,3,2500
3,4,Jagdish Jadhav,3,4000
4,5,Mahendra Saha,3,2200
5,6,Harishankar Nagar,6,4000
6,7,Kavita,3,3850
7,8,Abhinav,3,2250
8,9,Dhankesh Meena,6,4050
9,10,Gajananda Pareek,3,4100


## Creating Essential Dataframes from the Customer Data
- attrition rate for month x  = #left during x / # started at the start of x 


In [5]:
def attrition(data):
    '''
    This function returns the size of customer base, present matrix,count of monthly present, left, attrition rates
    '''

    a = []
    for i in data:
        a.append(data[i]['Membership Id'].max())
        b= max(a)
    
    present = pd.DataFrame(0, columns=data.keys(), index= np.arange(1,b+1))
    Sheet_names = list(data.keys())

    for i in range(len(data)):
        #print(i)
        data_1 = data[Sheet_names[i]][data[Sheet_names[i]]['Pkg_Duration'] == 1]
        for j in data_1['Membership Id']:
            present.iloc[j-1,i] = 1 

        data_2 = data[Sheet_names[i]][data[Sheet_names[i]]['Pkg_Duration'] == 2]
        for j in data_2['Membership Id']:
            present.iloc[j-1,i:i+2] = 1
        
        data_3 =  data[Sheet_names[i]][data[Sheet_names[i]]['Pkg_Duration'] == 3]
        for j in data_3['Membership Id']:
            present.iloc[j-1,i:i+3] = 1
        
        data_6 = data[Sheet_names[i]][data[Sheet_names[i]]['Pkg_Duration'] == 6]
        for j in data_6['Membership Id']:
            present.iloc[j-1,i:i+6] = 1
    
    left= []
    monthly_present = present.sum(axis=0)
    monthly_present_shifted = monthly_present.shift(1).drop('Aug_22')
    for i in range(1,len(data)):
        diff = present.iloc[:,i] - present.iloc[:,i-1]
        left.append(sum(diff<0))
    
    attrition_rates = left/monthly_present_shifted*100

    return(b,present,monthly_present,left,attrition_rates)
        

In [6]:
# calling the function
b,present,monthly_present,left,attrition_rates = attrition(data)

In [7]:
# size of the customer base
print("The size of the customer base is ",b)

The size of the customer base is  266


## Taking a Look at Output

### Present Matrix
- present matrix is the matrix that shows the presence of the customer in the gym
- 1 for active and 0 for inactive

In [8]:
print("The present matrix is \n",present)

The present matrix is 
      Aug_22  Sep_22  Oct_22  Nov_22  Dec_22  Jan_23  Feb_23  Mar_23  Apr_23  \
1         1       1       1       1       1       1       0       0       0   
2         1       1       1       1       1       1       0       0       0   
3         1       1       1       0       0       1       0       0       0   
4         1       1       1       0       0       1       0       0       0   
5         1       1       1       0       0       0       0       1       1   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
262       0       0       0       0       0       0       0       0       0   
263       0       0       0       0       0       0       0       0       0   
264       0       0       0       0       0       0       0       0       0   
265       0       0       0       0       0       0       0       0       0   
266       0       0       0       0       0       0       0       0       0   

     May_24  Jun_23  Jul_23

In [9]:
# Reseting the index of the present matrix
present.reset_index(inplace=True)
present.columns = ['Customer_Id','Aug_22','Sep_22','Oct_22','Nov_22','Dec_22','Jan_23','Feb_23','Mar_23','Apr_23','May_23','Jun_23','Jul_23','Aug_23']
present

,Customer_Id,Aug_22,Sep_22,Oct_22,Nov_22,Dec_22,Jan_23,Feb_23,Mar_23,Apr_23,May_23,Jun_23,Jul_23,Aug_23
0,1,1,1,1,1,1,1,0,0,0,0,0,0,0
1,2,1,1,1,1,1,1,0,0,0,0,0,0,0
2,3,1,1,1,0,0,1,0,0,0,0,0,0,0
3,4,1,1,1,0,0,1,0,0,0,0,0,0,0
4,5,1,1,1,0,0,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,262,0,0,0,0,0,0,0,0,0,0,0,0,1
262,263,0,0,0,0,0,0,0,0,0,0,0,0,1
263,264,0,0,0,0,0,0,0,0,0,0,0,0,1
264,265,0,0,0,0,0,0,0,0,0,0,0,0,1


In [10]:
# Saving the present matrix to a csv file
present.to_csv("Present.csv",index=False)  

### Monthly Present
- monthly present is the count of the customers present in the gym in a month

In [11]:
print("The monthly present is \n",monthly_present)

The monthly present is 
 Aug_22    46
Sep_22    67
Oct_22    66
Nov_22    70
Dec_22    61
Jan_23    83
Feb_23    64
Mar_23    59
Apr_23    44
May_24    29
Jun_23    10
Jul_23    14
Aug_23    19
dtype: int64


In [12]:
monthly_present=monthly_present.reset_index()
monthly_present.columns = ['Month','Active_Customers']
monthly_present

,Month,Active_Customers
0,Aug_22,46
1,Sep_22,67
2,Oct_22,66
3,Nov_22,70
4,Dec_22,61
5,Jan_23,83
6,Feb_23,64
7,Mar_23,59
8,Apr_23,44
9,May_24,29


In [13]:
monthly_present.to_csv("Monthly Present.csv", index=False)

### Attrition Rate
- It is a measure of the proportion of individuals or items moving out of a group over a specific period. 
- Attrition rate = (the number of customers who left a company and weren't replaced / the number of customers at the beginning of a specific time period) x 100.

In [14]:
# attrition rates
print("The attrition rates are \n",round(attrition_rates))

The attrition rates are 
 Sep_22    46.0
Oct_22    46.0
Nov_22    53.0
Dec_22    53.0
Jan_23    51.0
Feb_23    59.0
Mar_23    42.0
Apr_23    56.0
May_24    66.0
Jun_23    76.0
Jul_23    70.0
Aug_23    71.0
dtype: float64


In [15]:
# Converting to dataframe
attrition_rate=attrition_rates.reset_index()
attrition_rate.columns = ['Month','Attrition_Rate']
attrition_rate

,Month,Attrition_Rate
0,Sep_22,45.652174
1,Oct_22,46.268657
2,Nov_22,53.030303
3,Dec_22,52.857143
4,Jan_23,50.819672
5,Feb_23,59.036145
6,Mar_23,42.187500
7,Apr_23,55.932203
8,May_24,65.909091
9,Jun_23,75.862069


In [16]:
# saving the attrition rate to a csv file
attrition_rate.to_csv("Attrition Rate.csv",index=False)

# Customer Demographics Analysis

In [17]:
# cust_info file has the information regarding the customer demographics and etc
cust_info=pd.read_csv('cust_info1.csv')

# package_renewal file has the information regarding the package renewal data (in which month the customer has renewed the package)
renewal=pd.read_csv('all_renewals1.csv')

# Reading the newly created files
monthly_present=pd.read_csv('Monthly Present.csv')
attrition_rate=pd.read_csv('Attrition Rate.csv')
present=pd.read_csv('Present.csv')

## Customer Analysis

In [19]:
# Checking the first few rows of the cust_info file
cust_info.head()

,Customer_Id,Name,Address,Age,Gender,Package_type,Package_duration,Amount,Start_date,Attrition_flag,Start_month_year
0,1,Navin Sharma,Curaj,35.0,Male,Muscle,6.0,4500.0,22-08-2022,Existing,Aug-22
1,2,Nitesh yadav,Kishangarh,31.0,Male,Muscle,6.0,4500.0,22-08-2022,Attrited,Aug-22
2,3,Harishankar Shibu,Curaj,21.0,Male,Muscle,3.0,2500.0,22-08-2022,Attrited,Aug-22
3,4,Jagdish Jadhav,Curaj,47.0,Male,Both,3.0,4000.0,22-08-2022,Attrited,Aug-22
4,5,Mahendra Saha,Curaj,41.0,Male,Muscle,3.0,2200.0,22-08-2022,Existing,Aug-22


In [20]:
# Checking for the missing values
cust_info.isnull().sum()

Customer_Id          0
Name                 0
Address              4
Age                 13
Gender               0
Package_type         2
Package_duration     2
Amount               2
Start_date           1
Attrition_flag       0
Start_month_year     1
dtype: int64

In [21]:
# Checking for Duplicate values
cust_info.duplicated().sum()

0

## Gender

In [23]:
# Gender Value counts
gender_temp=cust_info.Gender.value_counts()
gender_temp

Male      235
Female     31
Name: Gender, dtype: int64

In [24]:
# Gender Distribution
gender_fig=px.pie(values=gender_temp,names=gender_temp.index,title="Gender Distribution")
gender_fig.show()

In [ ]:
# gender_fig.write_image("Gender.png")

## Age

In [25]:
# Age Distribution
age_fig=px.histogram(cust_info,x='Age',title="Age Distribution")
age_fig.show()

In [ ]:
# age_fig.write_image("Age.png")

In [26]:
# Statistical Summary of the Age
stats=cust_info['Age'].describe()
stats

count    253.000000
mean      23.727273
std        5.148656
min       16.000000
25%       21.000000
50%       23.000000
75%       26.000000
max       52.000000
Name: Age, dtype: float64

In [ ]:
# stats.to_csv('Age Stats.csv')

## Package Type

In [27]:
# Package Type Value Counts
pack_type_temp=cust_info.Package_type.value_counts()
pack_type_temp

Muscle    232
Both       32
Name: Package_type, dtype: int64

In [28]:
# Package Type Distribution
package_type_fig=px.pie(values=pack_type_temp,names=pack_type_temp.index,title="Package Type Distribution")
package_type_fig.show()

In [ ]:
# package_type_fig.write_image("Package Type.png")

## Package Duration

In [29]:
# Package Duration Value Counts
pack_duration_temp=cust_info.Package_duration.value_counts()

In [30]:
# Package Duration Distribution
package_duration_fig=px.pie(values=pack_duration_temp,names=pack_duration_temp.index,title="Package Duration Distribution")
package_duration_fig.show()

In [31]:
# package_duration_fig.write_image("Package Duration.png")

## Address

In [32]:
# Address have missing values. So, I will replace it with unknown
cust_info['Address'].fillna('Unknown',inplace=True)

In [33]:
cust_info.Address.value_counts()

Curaj             105
Curaj              81
Bandarsindari      41
Mundoti            15
Mundoti             7
Bandar seendri      7
Unknown             4
Bandarsindri        3
Kishangarh          1
Nohria              1
Cueaj               1
Name: Address, dtype: int64

- There are some values with the different spelling, so let's correct them.

In [37]:
# Replacing the misspelled values
cust_info.Address.replace(["Curaj ","Cueaj","Bandarsindri","Kishangarh ","Bandarsindari ","Mundoti ","Nohria ","Bandar seendri"],
                          ["Curaj","Curaj","Bandarsindari","Kishangarh","Bandarsindari","Mundoti","Nohria","Bandarsindari"],inplace=True)


In [38]:
# Address Value Counts
address_temp=cust_info.Address.value_counts()
address_temp

Curaj            187
Bandarsindari     51
Mundoti           22
Unknown            4
Kishangarh         1
Nohria             1
Name: Address, dtype: int64

In [39]:
# Address Distribution
address_fig=px.pie(values=address_temp,names=address_temp.index,title="Address Distribution")
address_fig.show()

In [40]:
# address_fig.write_image("Address.png")

# Revenue Analysis

In [41]:
# Pandas reads date as object, so we need to convert it to datetime format
cust_info["Start_month_year"] = pd.to_datetime(cust_info["Start_month_year"], format='%b-%y')
renewal["month_year"] = pd.to_datetime(renewal["month_year"], format='%b-%y')

In [42]:
# Revenue from New Registration
revenue_from_registration=cust_info.groupby("Start_month_year")["Amount"].sum().reset_index()
revenue_from_registration

,Start_month_year,Amount
0,2022-08-01,91700.0
1,2022-09-01,57200.0
2,2022-10-01,31450.0
3,2022-11-01,37380.0
4,2022-12-01,31300.0
5,2023-01-01,42450.0
6,2023-02-01,9600.0
7,2023-03-01,17800.0
8,2023-04-01,8800.0
9,2023-05-01,6400.0


In [43]:
# Revenue from Renewals
renewal_revenue=renewal.groupby("month_year")["Amt_Paid"].sum().reset_index()
renewal_revenue

,month_year,Amt_Paid
0,2022-09-01,700
1,2022-10-01,1900
2,2022-11-01,22550
3,2022-12-01,18900
4,2023-01-01,28500
5,2023-02-01,30650
6,2023-03-01,22600
7,2023-04-01,14900
8,2023-05-01,11300
9,2023-06-01,2200


In [44]:
# Merging the two dataframes to get the total revenue
revenue=pd.merge(revenue_from_registration,renewal_revenue,right_on="month_year",left_on="Start_month_year",how="outer").drop("month_year",axis=1)
revenue

,Start_month_year,Amount,Amt_Paid
0,2022-08-01,91700.0,NaN
1,2022-09-01,57200.0,700.0
2,2022-10-01,31450.0,1900.0
3,2022-11-01,37380.0,22550.0
4,2022-12-01,31300.0,18900.0
5,2023-01-01,42450.0,28500.0
6,2023-02-01,9600.0,30650.0
7,2023-03-01,17800.0,22600.0
8,2023-04-01,8800.0,14900.0
9,2023-05-01,6400.0,11300.0


In [45]:
# Renaming the columns for convenience
revenue.columns=["Date","from_registration","from_renewal"]
revenue

,Date,from_registration,from_renewal
0,2022-08-01,91700.0,NaN
1,2022-09-01,57200.0,700.0
2,2022-10-01,31450.0,1900.0
3,2022-11-01,37380.0,22550.0
4,2022-12-01,31300.0,18900.0
5,2023-01-01,42450.0,28500.0
6,2023-02-01,9600.0,30650.0
7,2023-03-01,17800.0,22600.0
8,2023-04-01,8800.0,14900.0
9,2023-05-01,6400.0,11300.0


In [46]:
# Filling the missing values with 0
revenue.from_registration.fillna(0,inplace=True)
revenue.from_renewal.fillna(0,inplace=True)
revenue

,Date,from_registration,from_renewal
0,2022-08-01,91700.0,0.0
1,2022-09-01,57200.0,700.0
2,2022-10-01,31450.0,1900.0
3,2022-11-01,37380.0,22550.0
4,2022-12-01,31300.0,18900.0
5,2023-01-01,42450.0,28500.0
6,2023-02-01,9600.0,30650.0
7,2023-03-01,17800.0,22600.0
8,2023-04-01,8800.0,14900.0
9,2023-05-01,6400.0,11300.0


In [47]:
# Total Revenue
revenue["Total_revenue"]=revenue["from_registration"]+revenue["from_renewal"]
revenue

,Date,from_registration,from_renewal,Total_revenue
0,2022-08-01,91700.0,0.0,91700.0
1,2022-09-01,57200.0,700.0,57900.0
2,2022-10-01,31450.0,1900.0,33350.0
3,2022-11-01,37380.0,22550.0,59930.0
4,2022-12-01,31300.0,18900.0,50200.0
5,2023-01-01,42450.0,28500.0,70950.0
6,2023-02-01,9600.0,30650.0,40250.0
7,2023-03-01,17800.0,22600.0,40400.0
8,2023-04-01,8800.0,14900.0,23700.0
9,2023-05-01,6400.0,11300.0,17700.0


In [48]:
# Plotting the Revenue from Registration and Renewal
revenue_fig=px.line(revenue,x="Date",y=["from_registration","from_renewal","Total_revenue"],
                    title="Revenue from Registration and Renewal",
                    labels=dict(variable="Revenue",Date="Date",value="Revenue"))
revenue_fig.show()

In [49]:
# revenue_fig.write_image("Revenue.png")

### Analysing the contribution of different time periods

In [50]:
# Total Revenue Collection
total_revenue_collection=revenue.Total_revenue.sum()
total_revenue_collection

521780.0

In [51]:
# First four months total revenue
first_four_months_revenue = revenue.iloc[:4]['Total_revenue'].sum()

# Next four months total revenue
next_four_months_revenue = revenue.iloc[4:8]['Total_revenue'].sum()

# Last five months total revenue
last_five_months_revenue = revenue.iloc[-5:]['Total_revenue'].sum()

# Create dataframe
revenue_summary = pd.DataFrame({'Time Period': ['Aug-22 to Nov-22', 'Dec-22 to Mar-23', 'Apr-23 to Aug-23'],
                                'Total Revenue': [first_four_months_revenue, next_four_months_revenue, last_five_months_revenue]})

revenue_summary['Percentage'] = round(revenue_summary['Total Revenue']/total_revenue_collection*100,1)
revenue_summary


,Time Period,Total Revenue,Percentage
0,Aug-22 to Nov-22,242880.0,46.5
1,Dec-22 to Mar-23,201800.0,38.7
2,Apr-23 to Aug-23,77100.0,14.8


In [52]:
fig = px.pie(revenue_summary, values='Total Revenue', names='Time Period',hole=0.3, title='Revenue Collection Summary')
fig.update_layout(width=350, height=300,margin=dict(l=20, r=20, t=50, b=0))
fig.show()


In [53]:
# fig.write_image("Revenue Summary.png")

# Attrtion Analysis

In [54]:
# Creating a new df with some categorical columns from sutomer info
customer_temp=cust_info.loc[:,["Customer_Id","Gender","Package_type","Address"]]
customer_temp

,Customer_Id,Gender,Package_type,Address
0,1,Male,Muscle,Curaj
1,2,Male,Muscle,Kishangarh
2,3,Male,Muscle,Curaj
3,4,Male,Both,Curaj
4,5,Male,Muscle,Curaj
...,...,...,...,...
261,262,Male,Muscle,Bandarsindari
262,263,Male,Muscle,Bandarsindari
263,264,Male,Muscle,Bandarsindari
264,265,Male,Muscle,Mundoti


In [55]:
# present is a df with all the customers who are present in the current month
present

,Customer_Id,Aug_22,Sep_22,Oct_22,Nov_22,Dec_22,Jan_23,Feb_23,Mar_23,Apr_23,May_23,Jun_23,Jul_23,Aug_23
0,1,1,1,1,1,1,1,0,0,0,0,0,0,0
1,2,1,1,1,1,1,1,0,0,0,0,0,0,0
2,3,1,1,1,0,0,1,0,0,0,0,0,0,0
3,4,1,1,1,0,0,1,0,0,0,0,0,0,0
4,5,1,1,1,0,0,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,262,0,0,0,0,0,0,0,0,0,0,0,0,1
262,263,0,0,0,0,0,0,0,0,0,0,0,0,1
263,264,0,0,0,0,0,0,0,0,0,0,0,0,1
264,265,0,0,0,0,0,0,0,0,0,0,0,0,1


In [56]:
# Merging the two dfs to get the customers who are present in the current month and their details
cust_present=pd.merge(customer_temp,present,on="Customer_Id",how="outer")
cust_present

,Customer_Id,Gender,Package_type,Address,Aug_22,Sep_22,Oct_22,Nov_22,Dec_22,Jan_23,Feb_23,Mar_23,Apr_23,May_23,Jun_23,Jul_23,Aug_23
0,1,Male,Muscle,Curaj,1,1,1,1,1,1,0,0,0,0,0,0,0
1,2,Male,Muscle,Kishangarh,1,1,1,1,1,1,0,0,0,0,0,0,0
2,3,Male,Muscle,Curaj,1,1,1,0,0,1,0,0,0,0,0,0,0
3,4,Male,Both,Curaj,1,1,1,0,0,1,0,0,0,0,0,0,0
4,5,Male,Muscle,Curaj,1,1,1,0,0,0,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,262,Male,Muscle,Bandarsindari,0,0,0,0,0,0,0,0,0,0,0,0,1
262,263,Male,Muscle,Bandarsindari,0,0,0,0,0,0,0,0,0,0,0,0,1
263,264,Male,Muscle,Bandarsindari,0,0,0,0,0,0,0,0,0,0,0,0,1
264,265,Male,Muscle,Mundoti,0,0,0,0,0,0,0,0,0,0,0,0,1


In [57]:
# Finding the customers who are active in the last month (Data collection month)
last_month_active=cust_present.loc[:,["Customer_Id","Gender","Package_type","Address","Aug_23"]]
last_month_active.rename(columns={"Aug_23":"Active"},inplace=True)
last_month_active.head()

,Customer_Id,Gender,Package_type,Address,Active
0,1,Male,Muscle,Curaj,0
1,2,Male,Muscle,Kishangarh,0
2,3,Male,Muscle,Curaj,0
3,4,Male,Both,Curaj,0
4,5,Male,Muscle,Curaj,0


## Monthly Active

In [58]:
# Monthe wise active customers
monthly_present

,Month,Active_Customers
0,Aug_22,46
1,Sep_22,67
2,Oct_22,66
3,Nov_22,70
4,Dec_22,61
5,Jan_23,83
6,Feb_23,64
7,Mar_23,59
8,Apr_23,44
9,May_24,29


In [59]:
# Plotting the monthly active customers
monthly_present_fig=px.bar(monthly_present,x="Month",y="Active_Customers",title="Monthly Active Customers",
                           text_auto=True,labels=dict(Month="Month",Active_Customers="Active Customers"))
monthly_present_fig.show()

In [60]:
# monthly_present_fig.write_image("Monthly Active Customers.png")

## Gender vs Active Customers

In [61]:
# Number of active customers in each month based on Gender column
gender_active=cust_present.groupby("Gender")[['Aug_22', 'Sep_22',
    'Oct_22', 'Nov_22', 'Dec_22', 'Jan_23', 'Feb_23', 'Mar_23', 'Apr_23',
    'May_23', 'Jun_23', 'Jul_23', 'Aug_23']].sum().transpose().reset_index()
gender_active.columns=["Month","Female","Male"]
gender_active


,Month,Female,Male
0,Aug_22,4,42
1,Sep_22,8,59
2,Oct_22,8,58
3,Nov_22,11,59
4,Dec_22,10,51
5,Jan_23,6,77
6,Feb_23,5,59
7,Mar_23,6,53
8,Apr_23,8,36
9,May_23,6,23


In [62]:
gender_active_fig=px.bar(gender_active,x="Month",y=["Female","Male"],title="Active Customers and Gender",text_auto=True,
                         labels=dict(variable="Gender",Month="Month",value="Total Active Customers"))
gender_active_fig.show()

In [63]:
# gender_active_fig.write_image("gender_active_fig.png")

In [64]:
# Contentingency table
gender_active_contingency=pd.crosstab(last_month_active.Gender,last_month_active.Active)
gender_active_contingency

Active,0,1
Gender,,
Female,30,1
Male,217,18


In [65]:
# Plotting the contingency table
fig=px.imshow(gender_active_contingency,text_auto=True)
fig.update_layout(width=350,height=300,title="Gender and Active Customers")

In [66]:
# fig.write_image("gender_active_contingency.png")

In [67]:
# Fisher's Exact Test for Independence
odds_ratio, p_value = fisher_exact(gender_active_contingency)

print("Odds Ratio:",odds_ratio)
print("P Value:",p_value)

Odds Ratio: 2.488479262672811
P Value: 0.7080261469455373


## Address vs Active Customers

In [68]:
# Number of active customers in each month based on Address column
address_active=cust_present.groupby("Address")[['Aug_22', 'Sep_22',
    'Oct_22', 'Nov_22', 'Dec_22', 'Jan_23', 'Feb_23', 'Mar_23', 'Apr_23',
    'May_23', 'Jun_23', 'Jul_23', 'Aug_23']].sum().transpose().reset_index()
address_active.columns=["Month","Bandarsindari","Curaj","Kishangarh","Mundoti","Nohria","Unknown"]
address_active

,Month,Bandarsindari,Curaj,Kishangarh,Mundoti,Nohria,Unknown
0,Aug_22,10,28,1,7,0,0
1,Sep_22,20,40,1,6,0,0
2,Oct_22,17,42,1,6,0,0
3,Nov_22,12,50,1,6,1,0
4,Dec_22,4,49,1,6,1,0
5,Jan_23,5,75,1,1,0,1
6,Feb_23,6,50,0,5,1,2
7,Mar_23,4,49,0,5,1,0
8,Apr_23,2,39,0,2,1,0
9,May_23,5,20,0,3,1,0


In [69]:
address_active_fig=px.bar(address_active,x="Month",y=["Bandarsindari","Curaj","Kishangarh","Mundoti","Nohria","Unknown"],title="Active Customers and Address",text_auto=True,
                          labels=dict(variable="Address",Month="Month",value="Total Active Customers"))
address_active_fig.show()

In [70]:
# address_active_fig.write_image("address_active_fig.png")

**Proceeding with test**

In [72]:
# Creating a subset of the last_month_active df for the address and active columns
address_active_relation=last_month_active.loc[:,["Address","Active"]]

# Unique values in the Address column
address_active_relation.Address.unique()

array(['Curaj', 'Kishangarh', 'Bandarsindari', 'Mundoti', 'Nohria',
       'Unknown'], dtype=object)

In [73]:
# Merging Some of the values in the Address column to fulfill the assumption of the chi-square test
address_active_relation.replace(["Bandarsindari","Kishangarh","Mundoti","Nohria","Unknown"],"Others",inplace=True)
address_active_relation.Address.unique()

array(['Curaj', 'Others'], dtype=object)

In [74]:
# Contingency table
address_active_contingency=pd.crosstab(address_active_relation.Address,address_active_relation.Active)
address_active_contingency

Active,0,1
Address,,
Curaj,179,8
Others,68,11


In [75]:
# Contingency Plot
fig=px.imshow(address_active_contingency,text_auto=True)
fig.update_layout(title="Address and Active Customers",width=350,height=300)

In [76]:
# fig.write_image("address_active_contingency.png")

In [77]:
# Chi-square test for independence
statistics,pvalue,dof,expected=chi2_contingency(address_active_contingency)
print("Chi-square statistic:",statistics)
print("P-value:",pvalue)
print("Degrees of freedom:",dof)

Chi-square statistic: 6.404532176684075
P-value: 0.011382941560970755
Degrees of freedom: 1


## Package Type vs Active Customers

In [78]:
# Number of active customers in each month based on Package_type column
package_active=cust_present.groupby("Package_type")[['Aug_22', 'Sep_22',
    'Oct_22', 'Nov_22', 'Dec_22', 'Jan_23', 'Feb_23', 'Mar_23', 'Apr_23',
    'May_23', 'Jun_23', 'Jul_23', 'Aug_23']].sum().transpose().reset_index()
package_active.columns=["Month","Both","Muscle"]
package_active


,Month,Both,Muscle
0,Aug_22,8,38
1,Sep_22,14,53
2,Oct_22,11,55
3,Nov_22,12,58
4,Dec_22,9,52
5,Jan_23,10,73
6,Feb_23,4,60
7,Mar_23,8,51
8,Apr_23,6,38
9,May_23,3,26


In [81]:
# Plotting the active customers and package type
package_active_fig=px.bar(package_active,x="Month",y=["Both","Muscle"],title="Active Customers and Package Type",text_auto=True,
                          labels=dict(variable="Package Type",Month="Month",value="Total Active Customers"))
package_active_fig.show()

In [82]:
# package_active_fig.write_image("package_active_fig.png")

**Proceeding with test**

In [83]:
# Contingency table
package_active_contingency=pd.crosstab(last_month_active.Package_type,last_month_active.Active)
package_active_contingency

Active,0,1
Package_type,,
Both,31,1
Muscle,214,18


In [84]:
# Contingency Plot
fig=px.imshow(package_active_contingency,text_auto=True)
fig.update_layout(title="Package Type and Active Customers",width=350,height=300)
fig.show()

In [85]:
# fig.write_image("package_active_contingency.png")

In [86]:
# Fisher's Exact Test for Independence
odds_ratio, p_value = fisher_exact(package_active_contingency)

print("Odds Ratio:",odds_ratio)
print("P Value:",p_value)

Odds Ratio: 2.607476635514019
P Value: 0.48508917002283386


## Attrition Rate

In [87]:
attrition_rate

,Month,Attrition_Rate
0,Sep_22,45.652174
1,Oct_22,46.268657
2,Nov_22,53.030303
3,Dec_22,52.857143
4,Jan_23,50.819672
5,Feb_23,59.036145
6,Mar_23,42.187500
7,Apr_23,55.932203
8,May_24,65.909091
9,Jun_23,75.862069


In [90]:
# Plotting the attrition rate
fig=px.bar(attrition_rate,x="Month",y="Attrition_Rate",title="Attrition Rate",text_auto=True)
fig.show()

In [91]:
# fig.write_image("Attrition Rate.png")

# Customer Feedback

In [92]:
# Reading the customer feedback file
cust_feedback=pd.read_csv('G:\BDM PROJECT\Analysis\Customer Feedback.csv')

In [93]:
cust_feedback.head()

,Name,Age,Gender,How long have you been a member of the gym? (In months),Reason for joining?,Reason for Leaving? (Choose all if left multiple times),Additional Comments,Rate Your Overall Experience,Were you satisfied with the gym services when you first joined?,What features of the gym did you like the most?,...,Rate the Communication from the management,"Were there specific services or amenities that you think were lacking? If yes, mention them.","Are you aware of other gym options in the area, including the college gym?","Have you tried the other gym options, if yes, what influenced you to try them?",Did you feel the concerns and feedbacks were adequately addressed?,What improvements would you suggest to enhance the overall experience?,Are there specific services or features you would like to see added?,How likely are you to consider rejoining the gym in the future?,Is there anything else you want to share about your experience with the gym?,"If you have joined another gym in the area, what are the negative points about that gym?"
0,Debjyoti Mukherjee,25,Male,6,Basic fitness,Never left,NaN,5,Yes,"Equipment Quality, Environment",...,4,None,Yes,No,Yes,Competitive pricing,Treadmills being available to all users irresp...,5,NaN,NaN
1,kshitiz meena,22,Male,3 month,Basic fitness,"Membership cost, Not satisfied with equipments...",if this university is private then i will pay ...,2,No,"Equipment Quality, cleanliness",...,2,some equipment are missing,No,firstly the college gym which is free ... but ...,No,reduce cost provide better trainer,no,4,it was good but cost is high for sure,"Timming, Equipment, relatively timing issues a..."
2,DISHANT ROHILA,24,Male,2,Strength training,Moved to another location,Nice gym,3,Yes,Environment,...,3,More space,Yes,Renewal of the university gym,Yes,NaN,More equipment,1,NaN,NaN
3,Sharvan,23,Male,4 months,Strength training,Availability of other options,NaN,4,Yes,"cleanliness, Timings",...,4,No,Yes,"Yes,availability of gym inside University",Yes,NaN,Calf raise seated machine,5,Thank you very much for a great training exper...,"Crowd, Timming"
4,Ankita Bhuparia,24,Female,6,Fat Loss Journey,"Membership cost, Not satisfied with equipments...",NaN,3,Yes,"cleanliness, Timings",...,2,No,Yes,Yes,No,NaN,Ni,1,NaN,"Crowd, Timming"


In [94]:
# Customer Feedback Questions
cust_feedback.columns

Index(['Name', 'Age', 'Gender ',
       'How long have you been a member of the gym? (In months)',
       'Reason for joining?',
       'Reason for Leaving? (Choose all if left multiple times)',
       'Additional Comments', 'Rate Your Overall Experience ',
       'Were you satisfied with the gym services when you first joined?',
       'What features of the gym did you like the most?',
       'What features of the gym did you dislike the most?',
       'Rate the equipment', 'Rate the Pricing',
       'Rate the membership packages ', 'Rate the renewal process',
       'Rate the Trainers', 'Rate the Cleanliness and sanitary facilities',
       'Rate the Communication from the management',
       'Were there specific services or amenities that you think were lacking? If yes, mention them.',
       'Are you aware of other gym options in the area, including the college gym?',
       'Have you tried the other gym options, if yes, what influenced you to try them?',
       'Did you feel the c

## Reason for Joining

In [95]:
joining_reason=cust_feedback.loc[:,"Reason for joining?"].value_counts().reset_index()
joining_reason.columns=["Reason","Count"]
joining_reason["Percentage"]=(joining_reason["Count"]/joining_reason["Count"].sum())*100
joining_reason

,Reason,Count,Percentage
0,Basic fitness,10,50.0
1,Strength training,6,30.0
2,Fat Loss Journey,2,10.0
3,Bulking,2,10.0


In [96]:
fig=px.pie(joining_reason,values="Count",names="Reason",title="Reason for Joining",hole=0.3)
# adjust the dimensions of the figure
fig.update_layout(width=350,height=300,title_x=0.5,margin=dict(l=20, r=20, t=50, b=0))
fig.show()

In [97]:
fig.write_image("joining_reason.png")

## Reason for leaving

In [98]:
leaving_reason =cust_feedback.loc[:,"Reason for Leaving? (Choose all if left multiple times)"]
leaving_reason

0                                            Never left
1     Membership cost, Not satisfied with equipments...
2                             Moved to another location
3                         Availability of other options
4     Membership cost, Not satisfied with equipments...
5     Not satisfied with equipments or services,Avai...
6                                          Lack of time
7        Membership cost, Availability of other options
8                                          Lack of time
9                                          Lack of time
10                                         Lack of time
11                                         Lack of time
12                                         Lack of time
13                        Lack of time, Membership cost
14                            Moved to another location
15                                         Lack of time
16                            Moved to another location
17    Membership cost, Not satisfied with equipm

In [100]:
# Adding all reasons to a list
leaving_reason_list=[]
for i in leaving_reason:
    leaving_reason_list.extend(i.split(","))


leaving_reason_series=pd.Series(leaving_reason_list)
leaving_reason_series


0                                     Never left
1                                Membership cost
2      Not satisfied with equipments or services
3                  Availability of other options
4                      Moved to another location
5                  Availability of other options
6                                Membership cost
7      Not satisfied with equipments or services
8      Not satisfied with equipments or services
9                  Availability of other options
10                                  Lack of time
11                               Membership cost
12                 Availability of other options
13                                  Lack of time
14                                  Lack of time
15                                  Lack of time
16                                  Lack of time
17                                  Lack of time
18                                  Lack of time
19                               Membership cost
20                  

In [101]:
# Unique values in the leaving reason
leaving_reason_series.unique()

array(['Never left', 'Membership cost',
       ' Not satisfied with equipments or services',
       'Availability of other options', 'Moved to another location',
       'Not satisfied with equipments or services', 'Lack of time',
       ' Availability of other options', ' Membership cost'], dtype=object)

In [102]:
# Replacing the misspelled values
leaving_reason_series.replace([ " Not satisfied with equipments or services"," Membership cost"," Availability of other options"],
[ "Not satisfied with equipments or services","Membership cost","Availability of other options"],inplace=True)

In [103]:
#print(leaving_reason_series.unique())
print(leaving_reason_series.value_counts())

Lack of time                                 9
Membership cost                              6
Not satisfied with equipments or services    5
Availability of other options                5
Moved to another location                    4
Never left                                   1
dtype: int64


In [104]:
# Calculating the percentage of each reason
leaving_reason_df=leaving_reason_series.value_counts().reset_index()
leaving_reason_df.columns=["Reason","Count"]
leaving_reason_df["Percentage"]=round(leaving_reason_df["Count"]/leaving_reason_df["Count"].sum()*100,2)
leaving_reason_df

,Reason,Count,Percentage
0,Lack of time,9,30.00
1,Membership cost,6,20.00
2,Not satisfied with equipments or services,5,16.67
3,Availability of other options,5,16.67
4,Moved to another location,4,13.33
5,Never left,1,3.33


In [105]:
fig=px.bar(leaving_reason_df,x="Reason",y="Percentage",text_auto=True)
fig.update_layout(margin=dict(l=20,r=20,t=0,b=0),xaxis=dict(tickangle=10))
fig.show()

In [106]:
# fig.write_image("leaving_reason.png")

In [107]:
fig=px.pie(leaving_reason_df,values="Count",names="Reason",title="Reason for Leaving",hole=0.3)
# adjust the dimensions of the figure
fig.update_layout(width=550,height=350,title_x=0.5,margin=dict(l=20,r=20,t=40,b=0))
fig.show()

In [108]:
# fig.write_image("leaving_reason_pie.png")

## Rate Your Overall Experience 

In [109]:
overall_exp=cust_feedback.loc[:,"Rate Your Overall Experience "].value_counts().reset_index()
overall_exp.columns=["Experience","Count"]
overall_exp["Percentage"]=round(overall_exp["Count"]/overall_exp["Count"].sum()*100,2)
overall_exp

,Experience,Count,Percentage
0,4,10,50.0
1,3,6,30.0
2,5,3,15.0
3,2,1,5.0


In [110]:
fig=px.pie(overall_exp,values="Count",names="Experience")
# adjust the dimensions of the figure
fig.update_layout(width=400,height=400)
fig.show()

In [111]:
# fig.write_image("overall_exp.png")

## Were you satisfied with the gym services when you first joined?

In [112]:
customer_satisfaction=cust_feedback.loc[:,"Were you satisfied with the gym services when you first joined?"].value_counts().reset_index()
customer_satisfaction.columns=["Satisfaction","Count"]
customer_satisfaction

,Satisfaction,Count
0,Yes,16
1,No,4


In [113]:
fig=px.pie(customer_satisfaction,values="Count",names="Satisfaction",title="Overall Experience")
# adjust the dimensions of the figure
fig.update_layout(width=400,height=400)
fig.show()

## What features of the gym did you like the most?

In [114]:
cust_feedback.loc[:,"What features of the gym did you like the most?"]

0                        Equipment Quality, Environment
1                        Equipment Quality, cleanliness
2                                           Environment
3                                  cleanliness, Timings
4                                  cleanliness, Timings
5                            Equipment Quality, Timings
6                                     Equipment Quality
7              Equipment Quality, Trainers, cleanliness
8                           Equipment Quality, Trainers
9     Equipment Quality, Trainers, cleanliness, Timi...
10                                    Equipment Quality
11    Equipment Quality, Trainers, cleanliness, Timi...
12                             cleanliness, Environment
13             Equipment Quality, Trainers, cleanliness
14    Equipment Quality, cleanliness, Timings, Envir...
15                                 Timings, Environment
16    Equipment Quality, cleanliness, Timings, Envir...
17                                    Equipment 

In [116]:
# Adding all the multiple values to a list
gym_features_likes_list=[]
for i in cust_feedback.loc[:,"What features of the gym did you like the most?"]:
    gym_features_likes_list.extend(i.split(","))

gym_features_likes_series=pd.Series(gym_features_likes_list)
gym_features_likes_series

0     Equipment Quality
1           Environment
2     Equipment Quality
3           cleanliness
4           Environment
5           cleanliness
6               Timings
7           cleanliness
8               Timings
9     Equipment Quality
10              Timings
11    Equipment Quality
12    Equipment Quality
13             Trainers
14          cleanliness
15    Equipment Quality
16             Trainers
17    Equipment Quality
18             Trainers
19          cleanliness
20              Timings
21          Environment
22    Equipment Quality
23    Equipment Quality
24             Trainers
25          cleanliness
26              Timings
27          Environment
28          cleanliness
29          Environment
30    Equipment Quality
31             Trainers
32          cleanliness
33    Equipment Quality
34          cleanliness
35              Timings
36          Environment
37              Timings
38          Environment
39    Equipment Quality
40          cleanliness
41              

In [117]:
# Unique values in the gym features likes
gym_features_likes_series.unique()

array(['Equipment Quality', ' Environment', ' cleanliness', 'Environment',
       'cleanliness', ' Timings', ' Trainers', 'Timings'], dtype=object)

In [118]:
# Replacing the misspelled values
gym_features_likes_series.replace([" Environment"," cleanliness"," Timings"," Trainers","cleanliness"],[ "Environment","Cleanliness","Timings","Trainers","Cleanliness"],inplace=True)
gym_features_likes_series.value_counts()

Equipment Quality    15
Cleanliness          12
Environment           9
Timings               8
Trainers              6
dtype: int64

In [119]:
# Value counts
gym_features_likes_df=gym_features_likes_series.value_counts().reset_index()
gym_features_likes_df.columns=["Features","Count"]
gym_features_likes_df

,Features,Count
0,Equipment Quality,15
1,Cleanliness,12
2,Environment,9
3,Timings,8
4,Trainers,6


In [120]:
px.bar(gym_features_likes_df,x="Features",y="Count",title="Features of the gym liked the most",text_auto=True)

## Rate the Pricing

In [121]:
price_rating =cust_feedback.loc[:,"Rate the Pricing"].value_counts().reset_index()
price_rating.columns=["Rating","Count"]
price_rating["Percentage"]=(price_rating["Count"]/price_rating["Count"].sum())*100
price_rating


,Rating,Count,Percentage
0,3,6,30.0
1,2,5,25.0
2,4,4,20.0
3,5,3,15.0
4,1,2,10.0


In [122]:
fig=px.pie(price_rating,values="Count",names="Rating",title="Price Rating",hole=0.3)
# adjust the dimensions of the figure
fig.update_layout(width=400,height=400)
fig.show()

In [123]:
fig=px.bar(price_rating,x="Rating",y="Percentage",title="Price Rating",text_auto=True)
fig.update_layout(height=400,width=400)
fig.show()


In [125]:
# fig.write_image("price_rating.png")

## Rate the Trainers

In [126]:
trainer_rating = cust_feedback.loc[:,"Rate the Trainers"].value_counts().reset_index()
trainer_rating.columns=["Rating","Count"]
trainer_rating['Percentage'] = (trainer_rating['Count'] / trainer_rating['Count'].sum()) * 100
trainer_rating

,Rating,Count,Percentage
0,4,7,35.0
1,3,5,25.0
2,5,4,20.0
3,1,3,15.0
4,2,1,5.0


In [127]:
fig=px.bar(trainer_rating, x='Rating', y='Percentage', text='Percentage',
             labels={'Rating': 'Rating', 'Percentage': 'Percentage'},
             title='Distribution of Trainer Ratings with Percentages')
# center the title
fig.update_layout(title_x=0.5)

In [128]:
px.bar(trainer_rating,x="Rating",y="Count",title="Trainer Rating")

## Rate the Cleanliness and sanitary facilities

In [129]:
cleanliness_rating = cust_feedback.loc[:,"Rate the Cleanliness and sanitary facilities"].value_counts().reset_index()
cleanliness_rating.columns=["Rating","Count"]
cleanliness_rating['Percentage'] = round((cleanliness_rating['Count'] / cleanliness_rating['Count'].sum()) * 100,2)
cleanliness_rating

,Rating,Count,Percentage
0,4,11,55.0
1,5,5,25.0
2,3,4,20.0


In [ ]:
fig=px.bar(cleanliness_rating, x='Rating', y='Percentage', text_auto=True,title='Cleanliness Ratings with Percentages')
fig.show()

In [130]:
# fig.write_image("Cleanliness_Ratings.png")

## Rate the Communication from the management

In [131]:
communication_rating=cust_feedback.loc[:,"Rate the Communication from the management"].value_counts().reset_index()
communication_rating.columns=["Rating","Count"]
communication_rating['Percentage'] = round((communication_rating['Count'] / communication_rating['Count'].sum()) * 100)
communication_rating

,Rating,Count,Percentage
0,3,7,35.0
1,4,5,25.0
2,5,5,25.0
3,2,3,15.0


In [132]:
fig=px.bar(communication_rating, x='Rating', y='Percentage', text_auto=True,title='Communication Ratings with Percentages')
fig.show()

In [133]:
# fig.write_image("communication_rating.png")

## Were there specific services or amenities that you think were lacking? If yes, mention them.

In [ ]:
list(cust_feedback.loc[:,"Were there specific services or amenities that you think were lacking? If yes, mention them."])

['None',
 'some equipment are missing ',
 'More space ',
 'No',
 'No',
 'Squat rack and heavy dumbbell and incline bench more incline which cause shoulder injury ',
 ' value for money gym',
 'No',
 nan,
 'Nothing\n',
 'Yes, An area for warm-up and cool down exercises ',
 'L',
 'Area',
 'There are only two set of 2.5 and 5 kg of dumble so need to upgrade of dumble with proper weight ',
 "I don't think there was any ",
 'No',
 'Changing room and washroom',
 'Yes,trainer and good quality equipments.',
 'Hot water and cold water cooler machine ',
 nan]